In [0]:
import pandas
df = pandas.read_csv('songdata.csv')
df = df.drop(['link'], axis=1)

In [0]:
artistList =[]
count=0
for artist in df['artist']:
    if artist not in artistList:
        artistList.append(artist)

In [0]:
import math
import re
from collections import Counter

WORD = re.compile(r'\w+')

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

In [0]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist


def clearText(String):
    
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') #\b: Matches only the beginning or end of the word.
    text = re.sub(r'\W+' ,' ',String)  # for symbol
    text = re.sub(r'\d+',' ',text)   # digits
    text = pattern.sub(' ',text.lower())
    text = re.sub(r'\s+',' ',text)  #for space 
    tokenize = word_tokenize(text) 
    return(tokenize)

def process(singerName):

    lyrics = df.iloc[:,[0, -1]]
    lyrics = {k:''.join(g["text"].tolist()) for k,g in lyrics.groupby("artist")}

    songs = str(lyrics[singerName])
    tokenize= clearText(songs)
    
    fdist = FreqDist(tokenize)
    max = fdist.most_common(1)
  
    # conver to weight
    for x in fdist:
        fdist[x]=round(fdist[x]/max[0][1],2)

    # Product sentence scores 
    sent2score={}
    sent2score = {sentence : fdist[x] if sentence not in sent2score.keys() else sent2score[sentence] + fdist[x]  
    for sentence in songs.split('\n') for x in sentence.split(' ') if x in fdist.keys() }

    top_common = dict(Counter(sent2score).most_common(5)).keys()
    return([*top_common])

	

In [0]:
esasText = process('Queen')
artistList.remove('Queen')
list = []
for x in artistList[0:100]:
    text = process(x)
    list.append(text)
list

[["'Cause I want to, oh  ",
  'You will find, oh oh oh  ',
  "It's a beautiful life, oh oh oh oh  ",
  'Such a dream, oh oh oh  ',
  'Nobody gonna slow me down, oh no,  '],
 ['I felt that smoosh and I said oh Mama.  ',
  'Why, oh why  ',
  'That butt you got is.....ooh, so corny!  ',
  'He got a D.W.I.  ',
  'Baby, baby, baby, I want to stuff all that cabbage you got  '],
 ['Cause what if I never love again?  ',
  'Let this be our lesson in love  ',
  'Cause what if I never love again?',
  "I best tidy up my head I'm the only one in love  ",
  "I'm the only one in love  "],
 ['When you got to be yourself  ',
  "And how I got to thinkin'  ",
  "Yeah I got the DT's  ",
  "Said, I ain't got you  ",
  'For giving up all that you got  '],
 ["I've got no rights and I've got no style, but love is blind.  ",
  'You and me, we live for love  ',
  'You and Me, we live for love  ',
  'These pages are the book of love  ',
  "You can't explain the joy and pain of crazy love  "],
 ['Ganun pa man pak

In [0]:
cosValues = {}
for sen in range(len(esasText)):
    for text in range(len(list)):
        for sentence in range(len(list[text])):
        
            text1= esasText[sen]
            text2= list[text][sentence]
            text1=text1.lower()
            text2=text2.lower()

            vector1 = text_to_vector(text1)
            vector2 = text_to_vector(text2)  
            cosine=get_cosine(vector1,vector2)
            cosValues[text2]=cosine

In [0]:
most = Counter(cosValues).most_common(10)
most

[('i love you  ', 0.8660254037844387),
 ('i think i love you  ', 0.7559289460184544),
 ('carolina.......... i love you', 0.75),
 ('i will love you  ', 0.75),
 ("'cause i love you  ", 0.75),
 ('i love you dearly,  ', 0.75),
 ('where are you ? i love you . where are you ? i love you .  ',
  0.7071067811865475),
 ('i want you, need you, love you  ', 0.6933752452815365),
 ('you improve with time and i want you, i love you  ', 0.6882472016116852),
 ('oh, i love you so  ', 0.6708203932499369)]

In [0]:
song = [x[0] for x in most]

In [0]:
cleanedSong=[]
for t in range(len(song)):
    text = re.sub(r'\[[0-9]*\]',' ',song[t] )
    text = re.sub(r'\s+',' ',text)
    clean_text = text.lower()
    clean_text = re.sub(r'\W',' ',clean_text)
    clean_text = re.sub(r'\d',' ',clean_text)
    clean_text = re.sub(r'\s+',' ',clean_text)
    cleanedSong.append(clean_text)
cleanedSong

['i love you ',
 'i think i love you ',
 'carolina i love you',
 'i will love you ',
 ' cause i love you ',
 'i love you dearly ',
 'where are you i love you where are you i love you ',
 'i want you need you love you ',
 'you improve with time and i want you i love you ',
 'oh i love you so ']